In [2]:
import torch 

from pytorch_pretrained_bert import BertTokenizer,BertModel,BertForMaskedLM


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text = 'From Monday to Friday most people are busy working or studying, '\
       'but in the evenings and weekends they are free and _ themselves.'
tokenized_text = tokenizer.tokenize(text)

print(tokenized_text)

['from', 'monday', 'to', 'friday', 'most', 'people', 'are', 'busy', 'working', 'or', 'studying', ',', 'but', 'in', 'the', 'evenings', 'and', 'weekends', 'they', 'are', 'free', 'and', '_', 'themselves', '.']


In [8]:
masked_index = tokenized_text.index('_')

tokenized_text[masked_index] = '[MASK]'

print(tokenized_text)

['from', 'monday', 'to', 'friday', 'most', 'people', 'are', 'busy', 'working', 'or', 'studying', ',', 'but', 'in', 'the', 'evenings', 'and', 'weekends', 'they', 'are', 'free', 'and', '[MASK]', 'themselves', '.']


In [10]:
candidates = ['love', 'work', 'enjoy', 'play']

candidates_ids = tokenizer.convert_tokens_to_ids(candidates)

print("Candidates's ids are : ",candidates_ids )

Candidates's ids are :  [2293, 2147, 5959, 2377]


In [11]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

print('Indexed_tokens\'s ids are : ',indexed_tokens)

Indexed_tokens's ids are :  [2013, 6928, 2000, 5958, 2087, 2111, 2024, 5697, 2551, 2030, 5702, 1010, 2021, 1999, 1996, 16241, 1998, 13499, 2027, 2024, 2489, 1998, 103, 3209, 1012]


In [12]:
segments_ids = [0]*len(tokenized_text)

print(segments_ids)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [13]:
tokens_tensor = torch.tensor([indexed_tokens])

segments_tensors = torch.tensor([segments_ids])

print("Tokens's tensor: ",tokens_tensor)

print("Segments's tensor: ",segments_tensors)



Tokens's tensor:  tensor([[ 2013,  6928,  2000,  5958,  2087,  2111,  2024,  5697,  2551,  2030,
          5702,  1010,  2021,  1999,  1996, 16241,  1998, 13499,  2027,  2024,
          2489,  1998,   103,  3209,  1012]])
Segments's tensor:  tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]])


In [14]:
language_model = BertForMaskedLM.from_pretrained('bert-base-uncased')

language_model.eval()

print(language_model.eval())

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertInterm

In [15]:
predicitons = language_model(tokens_tensor, segments_tensors)

predictions_candidates = predicitons[0, masked_index, candidates_ids]

answer_idx = torch.argmax(predictions_candidates).item()

print("Predictions : ",predicitons)

print('-'*50)

print("Predictions_candidates:",predictions_candidates)

print('-'*50)

print(answer_idx)

Predictions :  tensor([[[ -7.3524,  -7.3705,  -7.3813,  ...,  -6.7036,  -6.6253,  -4.1512],
         [ -8.5638,  -9.0042,  -9.0043,  ...,  -6.2188,  -5.9293,  -5.9213],
         [-16.1689, -15.9357, -16.1867,  ..., -12.8809, -13.8796,  -4.9378],
         ...,
         [ -5.7302,  -6.0601,  -5.5552,  ...,  -5.6121,  -4.4892,  -7.0992],
         [ -8.7442,  -8.8843,  -8.7663,  ...,  -8.4347,  -7.4472,  -6.5295],
         [-10.8952, -10.9725, -10.8689,  ..., -10.7548, -10.0049,  -7.0908]]],
       grad_fn=<AddBackward0>)
--------------------------------------------------
Predictions_candidates: tensor([1.7453, 3.2638, 6.1241, 2.6489], grad_fn=<IndexBackward>)
--------------------------------------------------
2


In [16]:
print(f'The most likely word is "{candidates[answer_idx]}".')

The most likely word is "enjoy".
